In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [37]:
data=pd.read_csv("../data/datos_modificados.csv")
data=data.drop(columns="id")

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import GridSearchCV

In [38]:
data_usable=data[data["x_e_out [-]"].notnull()].copy()
prediction=data[data["x_e_out [-]"].isnull()].copy()

In [19]:
# Divido los datos en características (X) y columna objetivo (y)
X = data_usable.drop('x_e_out [-]', axis=1)
y = data_usable['x_e_out [-]']

# Identificar las columnas categóricas
categorical_cols = ['author', 'geometry']

# Definir el transformador para la codificación one-hot
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')

# Aplicar la codificación one-hot a las variables categóricas
X_encoded = ct.fit_transform(X)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, shuffle=True)

In [26]:
# Definir los hiperparámetros a ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Crear una instancia del modelo
model = RandomForestRegressor()

# Realizar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor estimador
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Entrenar el mejor modelo con los datos de entrenamiento completos
best_model.fit(X_train, y_train)

# Realizar predicciones con el mejor modelo
y_pred = best_model.predict(X_test)


In [29]:
print(best_model)

RandomForestRegressor(max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=10, n_estimators=200)


In [ ]:
#RandomForestRegressor(max_features='sqrt', min_samples_leaf=2,
#                      min_samples_split=10, n_estimators=200)

In [27]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 0.07474934975675115


In [39]:
prediction=prediction.drop(columns="x_e_out [-]")

In [41]:
prediction_encoded = ct.fit_transform(prediction)

In [43]:
prediccion_subir=best_model.predict(prediction_encoded)
prediccion_subir

array([ 0.01962778, -0.0844516 , -0.04568353, ...,  0.0370491 ,
       -0.04858809,  0.0045929 ])

In [47]:
indices=prediction.index

In [52]:
df = pd.DataFrame({'id': indices, 'x_e_out [-]': prediccion_subir})
df.to_csv("submissions/Submission1_model6.csv", index=False)